In [0]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split

In [0]:
from tensorflow import keras
from tensorflow.keras import models, layers
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model, save_model, clone_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, Callback

In [0]:
from google.colab import drive

In [0]:
import random
from PIL import ImageOps, ImageEnhance, ImageFilter, Image

In [0]:
np.random.seed(123)
random.seed(123)

In [0]:
image_size = (32, 32)
num_classes = 10

In [0]:
epochs = 200
batch_size = 128

# Load Data

In [0]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz
!rm cifar-10-python.tar.gz

--2019-05-05 00:21:43--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  5.09MB/s    in 23s     

2019-05-05 00:22:07 (6.94 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [0]:
data_dir = "cifar-10-batches-py/"

In [0]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict

# Load Train Data
train_data_dirs = [data_dir+"data_batch_{}".format(i) for i in range(1,6)]

raw_data = [unpickle(path) for path in train_data_dirs]
train_labels = np.array(sum([batch[b'labels'] for batch in raw_data], []))
train_data = np.vstack([batch[b'data'] for batch in raw_data])
train_data = np.moveaxis(train_data.reshape(-1,3,32,32),1,-1)

# Load Test Data
test_data_dir = data_dir+"test_batch"

raw_data = unpickle(test_data_dir)
test_labels = np.array(raw_data[b'labels'])
test_data = raw_data[b'data'] 
test_data = np.moveaxis(test_data.reshape(-1,3,32,32),1,-1)

# Load Label Names
label_names = unpickle(data_dir+"batches.meta")[b'label_names']

# Preprocess x and y 
X_train = [Image.fromarray((np.uint8(data))) for data in train_data]
X_train = train_data/ 255.
X_test = test_data/ 255.

y_train = np.eye(10)[train_labels]
y_test = np.eye(10)[test_labels]

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

# Make Model

In [0]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow(X_test, y_test)

validation_generator = test_datagen.flow(X_val, y_val)

In [5]:
# 0.8426
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow(X_train, y_train, 
                                      batch_size=batch_size)


model = models.Sequential()

model.add(layers.Conv2D(256, (3, 3), 
                        input_shape=(image_size[0], image_size[1], 3),
                        data_format='channels_last',
                        activation='elu', kernel_initializer='he_normal'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.BatchNormalization())


model.add(layers.Conv2D(256, (3, 3),
                        activation='elu',
                        kernel_initializer='he_normal'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, (2, 2),
                        activation='elu',
                        kernel_initializer='he_normal'))
model.add(layers.BatchNormalization())



model.add(layers.Flatten())


model.add(layers.Dense(128,
                       activation='elu',
                       kernel_initializer='he_normal'))
model.add(layers.BatchNormalization())

model.add(layers.Dense(64,
                       activation='elu',
                       kernel_initializer='he_normal'))
model.add(layers.BatchNormalization())

model.add(layers.Dense(num_classes, 
                       activation='softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 256)       7168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 256)       0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 15, 15, 256)       1024      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 256)       590080    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 6, 6, 256)         1024      
_________________________________________________________________
conv

In [0]:
checkpoint = ModelCheckpoint('checkpoint_model_%i.h5' % (model_num), 
                              monitor='val_loss', 
                              verbose=False, save_best_only=True, 
                              save_weights_only=True)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# Train

In [0]:
history = model.fit_generator(train_generator,
                    steps_per_epoch = len(X_train) // batch_size,
                    validation_data = validation_generator, 
                    validation_steps = len(X_val) // batch_size,
                    epochs=epochs, callbacks=[checkpoint])#, mixup])

Instructions for updating:
Use tf.cast instead.
Epoch 1/150
313/313 [==============================] - 55s 175ms/step - loss: 1.9393 - acc: 0.2978 - val_loss: 3.8302 - val_acc: 0.1893
Epoch 2/150
313/313 [==============================] - 47s 150ms/step - loss: 1.7549 - acc: 0.3550 - val_loss: 1.5590 - val_acc: 0.4141
Epoch 3/150
313/313 [==============================] - 47s 150ms/step - loss: 1.6103 - acc: 0.4152 - val_loss: 1.4690 - val_acc: 0.4803
Epoch 4/150
313/313 [==============================] - 47s 149ms/step - loss: 1.5004 - acc: 0.4575 - val_loss: 1.5420 - val_acc: 0.4572
Epoch 5/150
313/313 [==============================] - 47s 150ms/step - loss: 1.4222 - acc: 0.4890 - val_loss: 1.3522 - val_acc: 0.5136
Epoch 6/150
313/313 [==============================] - 47s 150ms/step - loss: 1.3437 - acc: 0.5189 - val_loss: 1.4212 - val_acc: 0.5182
Epoch 7/150
313/313 [==============================] - 47s 150ms/step - loss: 1.2810 - acc: 0.5439 - val_loss: 1.4820 - val_acc: 0.5178


In [0]:
model.save('fit_model_%i.h5' % (model_num))

In [0]:
model.load_weights('checkpoint_model_%i.h5' % (model_num))

In [0]:
model.save('loaded_model_%i.h5' % (model_num))

In [0]:
metrics = model.evaluate(X_test, y_test)

In [0]:
plt.figure(figsize=(14, 9))

plt.subplot(1, 2, 1)
plt.plot(range(len(model.history.history['loss'])), model.history.history['loss'], 
         label='Train Loss')
plt.plot(range(len(model.history.history['loss'])), model.history.history['val_loss'], 
         label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Cross entropy')
plt.text(0.6 * epochs, 0.75 * max(model.history.history['loss']),
         'Test loss = %.4f' %(metrics[0]))
plt.legend(loc='best')


plt.subplot(1, 2, 2)
plt.plot(range(len(model.history.history['loss'])), model.history.history['acc'], 
         label='Train Accuracy')
plt.plot(range(len(model.history.history['loss'])), model.history.history['val_acc'], 
         label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.text(0.6 * epochs, 0.8 * max(model.history.history['acc']),
         'Test Accuracy = %.4f' %(metrics[1]))
plt.legend(loc='best')


plt.savefig('training_metrics_%i.pdf' %(model_num))
plt.show()

# Test

In [0]:
classes = ['Plane', 'Car', 'Bird', 'Cat', 'Deer',
           'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
pred_sample = model.predict(X_test[:3])
preds = []
trues = []
for i in range(len(pred_sample)):
  p = list(pred_sample[i])
  y = list(y_test[i])
  this_pred = classes[p.index(max(p))]
  this_true = classes[y.index(max(y))]
  preds.append(this_pred)
  trues.append(this_true)
  print(this_pred, this_true)

In [0]:
plt.figure(figsize=(16, 8))

plt.subplot(1, 3, 1)
plt.title('Predicted %s, Actually %s' % (preds[0], trues[0]))
plt.imshow(X_test[0])

plt.subplot(1, 3, 2)
plt.title('Predicted %s, Actually %s' % (preds[1], trues[1]))
plt.imshow(X_test[1])

plt.subplot(1, 3, 3)
plt.title('Predicted %s, Actually %s' % (preds[2], trues[2]))
plt.imshow(X_test[2])

plt.savefig('prediction_sample_%i.pdf' % (model_num))
plt.show()

# Feature Visualization

In [0]:
# Get convolutional layer outputs
conv_layer_outputs = []
for layer in model.layers:
  if 'conv' in layer.name:
    conv_layer_outputs.append(layer.output)

In [0]:
output_model = models.Model(inputs=model.input, outputs=conv_layer_outputs)

In [0]:
activation_indices = np.random.randint(0, len(y_test), size=3)

In [0]:
activation_images = X_test[activation_indices]
activations = output_model.predict(activation_images)

In [0]:
plt.figure(figsize=(16, 12))

plt.subplot(4, 3, 1)
plt.imshow(activation_images[0])

plt.subplot(4, 3, 2)
plt.imshow(activation_images[1])

plt.subplot(4, 3, 3)
plt.imshow(activation_images[2])


plt.subplot(4, 3, 4)
plt.imshow(activations[0][0, :, :, 0])

plt.subplot(4, 3, 5)
plt.imshow(activations[0][1, :, :, 0])

plt.subplot(4, 3, 6)
plt.imshow(activations[0][2, :, :, 0])

plt.subplot(4, 3, 7)
plt.imshow(activations[0][0, :, :, 21])

plt.subplot(4, 3, 8)
plt.imshow(activations[0][1, :, :, 21])

plt.subplot(4, 3, 9)
plt.imshow(activations[0][2, :, :, 21])

plt.subplot(4, 3, 10)
plt.imshow(activations[0][0, :, :, 137])

plt.subplot(4, 3, 11)
plt.imshow(activations[0][1, :, :, 137])

plt.subplot(4, 3, 12)
plt.imshow(activations[0][2, :, :, 137])


plt.savefig('first_layer_activations_0_21_137_model%i.pdf' % (model_num))
plt.show()

In [0]:
plt.figure(figsize=(16, 12))

plt.subplot(4, 3, 1)
plt.imshow(activation_images[0])

plt.subplot(4, 3, 2)
plt.imshow(activation_images[1])

plt.subplot(4, 3, 3)
plt.imshow(activation_images[2])


plt.subplot(4, 3, 4)
plt.imshow(activations[0][0, :, :, 1])

plt.subplot(4, 3, 5)
plt.imshow(activations[0][1, :, :, 1])

plt.subplot(4, 3, 6)
plt.imshow(activations[0][2, :, :, 1])

plt.subplot(4, 3, 7)
plt.imshow(activations[0][0, :, :, 2])

plt.subplot(4, 3, 8)
plt.imshow(activations[0][1, :, :, 2])

plt.subplot(4, 3, 9)
plt.imshow(activations[0][2, :, :, 2])

plt.subplot(4, 3, 10)
plt.imshow(activations[0][0, :, :, 3])

plt.subplot(4, 3, 11)
plt.imshow(activations[0][1, :, :, 3])

plt.subplot(4, 3, 12)
plt.imshow(activations[0][2, :, :, 3])


plt.savefig('first_layer_activations_1to3_%i.pdf' %(model_num))
plt.show()

In [0]:
plt.figure(figsize=(16, 12))

plt.subplot(4, 3, 1)
plt.imshow(activation_images[0])

plt.subplot(4, 3, 2)
plt.imshow(activation_images[1])

plt.subplot(4, 3, 3)
plt.imshow(activation_images[2])


plt.subplot(4, 3, 4)
plt.imshow(activations[0][0, :, :, 4])

plt.subplot(4, 3, 5)
plt.imshow(activations[0][1, :, :, 4])

plt.subplot(4, 3, 6)
plt.imshow(activations[0][2, :, :, 4])

plt.subplot(4, 3, 7)
plt.imshow(activations[0][0, :, :, 5])

plt.subplot(4, 3, 8)
plt.imshow(activations[0][1, :, :, 5])

plt.subplot(4, 3, 9)
plt.imshow(activations[0][2, :, :, 5])

plt.subplot(4, 3, 10)
plt.imshow(activations[0][0, :, :, 6])

plt.subplot(4, 3, 11)
plt.imshow(activations[0][1, :, :, 6])

plt.subplot(4, 3, 12)
plt.imshow(activations[0][2, :, :, 6])


plt.savefig('first_layer_activations_4to6_%i.pdf' % (model_num))
plt.show()

In [0]:
plt.figure(figsize=(16, 12))

plt.subplot(4, 3, 1)
plt.imshow(activation_images[0])

plt.subplot(4, 3, 2)
plt.imshow(activation_images[1])

plt.subplot(4, 3, 3)
plt.imshow(activation_images[2])


plt.subplot(4, 3, 4)
plt.imshow(activations[1][0, :, :, 1])

plt.subplot(4, 3, 5)
plt.imshow(activations[1][1, :, :, 1])

plt.subplot(4, 3, 6)
plt.imshow(activations[1][2, :, :, 1])

plt.subplot(4, 3, 7)
plt.imshow(activations[1][0, :, :, 2])

plt.subplot(4, 3, 8)
plt.imshow(activations[1][1, :, :, 2])

plt.subplot(4, 3, 9)
plt.imshow(activations[1][2, :, :, 2])

plt.subplot(4, 3, 10)
plt.imshow(activations[1][0, :, :, 3])

plt.subplot(4, 3, 11)
plt.imshow(activations[1][1, :, :, 3])

plt.subplot(4, 3, 12)
plt.imshow(activations[1][2, :, :, 3])


plt.savefig('second_layer_activations_1to3_%i.pdf' % (model_num))
plt.show()

In [0]:
plt.figure(figsize=(16, 12))

plt.subplot(4, 3, 1)
plt.imshow(activation_images[0])

plt.subplot(4, 3, 2)
plt.imshow(activation_images[1])

plt.subplot(4, 3, 3)
plt.imshow(activation_images[2])


plt.subplot(4, 3, 4)
plt.imshow(activations[2][0, :, :, 1])

plt.subplot(4, 3, 5)
plt.imshow(activations[2][1, :, :, 1])

plt.subplot(4, 3, 6)
plt.imshow(activations[2][2, :, :, 1])

plt.subplot(4, 3, 7)
plt.imshow(activations[2][0, :, :, 2])

plt.subplot(4, 3, 8)
plt.imshow(activations[2][1, :, :, 2])

plt.subplot(4, 3, 9)
plt.imshow(activations[2][2, :, :, 2])

plt.subplot(4, 3, 10)
plt.imshow(activations[2][0, :, :, 3])

plt.subplot(4, 3, 11)
plt.imshow(activations[2][1, :, :, 3])

plt.subplot(4, 3, 12)
plt.imshow(activations[2][2, :, :, 3])


plt.savefig('third_layer_activations_1to3_%i.pdf' % (model_num))
plt.show()

In [0]:
# plt.close()
# plt.figure(figsize=(16, 16))

# plt.subplot(len(conv_layer_outputs) + 1, len(activation_images), 1)
# plt.imshow(activation_images[0])

# plt.subplot(len(conv_layer_outputs) + 1, len(activation_images), 2)
# plt.imshow(activation_images[1])

# plt.subplot(len(conv_layer_outputs) + 1, len(activation_images), 3)
# plt.imshow(activation_images[2])

# channel = 3
# for i in range(len(conv_layer_outputs)):
#   for j in range(len(activation_images)):
#     plt.subplot(len(conv_layer_outputs) + 1, len(activation_images), 
#                 len(activation_images) * (1 + i) + 1 + j)
#     print(len(activation_images) * (1 + i) + 1 + j)
# #     this_image = activations[i][j, :, :, channel]
#     plt.matshow(activations[i][j, :, :, channel], cmap='viridis')
# plt.show()

In [0]:
plt.figure(figsize=(16, 12))

plt.subplot(4, 3, 1)
plt.imshow(activation_images[0])

plt.subplot(4, 3, 2)
plt.imshow(activation_images[1])

plt.subplot(4, 3, 3)
plt.imshow(activation_images[2])


plt.subplot(4, 3, 4)
plt.imshow(activations[0][0, :, :, :3])

plt.subplot(4, 3, 5)
plt.imshow(activations[0][1, :, :, :3])

plt.subplot(4, 3, 6)
plt.imshow(activations[0][2, :, :, :3])

plt.savefig('stacked_first_layer_0to2_%i.pdf' % (model_num))
plt.show()